In [1]:
import os
import glob
import napari
import pandas as pd
import numpy as np
#import pyclesperanto_prototype as cle

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.processing.segment_and_quantify import segment_nuclei_cellpose, quantify
from blimp.preprocessing.illumination_correction import IlluminationCorrection

data_dir = "/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA"
image_dir = os.path.join(data_dir,"20240322_143358_441/OME-TIFF-MIP")

ana_dir = "/srv/scratch/z3532965/src/publications/2024_ARMC5/hek_polyA/"
illum_corr_file = os.path.join(data_dir,"/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/ILLUMCORR/illumination_correction.pkl")

01-Aug-24 11:39:50 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


# Apply illumination correction

Load illumination correction object

In [2]:
illumination_correction = IlluminationCorrection(from_file=illum_corr_file)

## Save examples for training nuclear segmentation model

In [3]:
training_dir = Path(image_dir).parent / 'TRAINING'
if not training_dir.exists(): 
    training_dir.mkdir()

In [4]:
example_image_files = glob.glob(str(Path(image_dir) / "*001.ome.tiff"))
example_image_files.sort()

In [5]:
for f in example_image_files[1:10]:
    aics_image = AICSImage(Path(image_dir) / Path(f).name, reader=readers.ome_tiff_reader.OmeTiffReader)
    aics_image_corrected = illumination_correction.correct(aics_image)
    # crop 500 pixels from left edge and save DAPI (C=1)
    DAPI = AICSImage(aics_image_corrected.get_image_data('YX',Z=0,C=1,T=0)[np.newaxis,np.newaxis,np.newaxis,:,500:])
    DAPI.save(str(training_dir / Path(f).name))

In [6]:
example_image_files

['/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/OME-TIFF-MIP/WellC03_Channel647,405,None,488_Seq0000_0001.ome.tiff',
 '/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/OME-TIFF-MIP/WellC04_Channel647,405,None,488_Seq0001_0001.ome.tiff',
 '/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/OME-TIFF-MIP/WellD03_Channel647,405,None,488_Seq0003_0001.ome.tiff',
 '/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/OME-TIFF-MIP/WellD04_Channel647,405,None,488_Seq0002_0001.ome.tiff',
 '/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/OME-TIFF-MIP/WellE03_Channel647,405,None,488_Seq0004_0001.ome.tiff',
 '/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/OME-TIFF-MIP/WellE04_Channel647,405,None,488_Seq0005_0001.ome

## Save examples for manual background measurements

In [7]:
examples_dir = Path(image_dir).parent / 'EXAMPLES'
if not examples_dir.exists(): 
    examples_dir.mkdir()

In [8]:
for f in example_image_files[1:10]:
    aics_image = AICSImage(Path(image_dir) / Path(f).name, reader=readers.ome_tiff_reader.OmeTiffReader)
    aics_image_corrected = illumination_correction.correct(aics_image)
    aics_image_corrected.save(examples_dir / Path(f).name)

In [9]:
examples_dir

PosixPath('/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240322/240322_ARMC5KO_PolyA/20240322_143358_441/EXAMPLES')